In [18]:
import sys
import importlib
import numpy as np
u_path = "../lib"
if not u_path in sys.path:
    sys.path.append(u_path)
import autoloader
importlib.reload(autoloader)
al = autoloader.Autoloader()

L90
N5
L180
L180
S4
F21
W4
S1
R270
F18
S4
F44
R90
N5
F18
E5
R270
F1
L90
W5
N4
W5
F37
R90
S1
W2
S1
L90
W5
R90
N1
W2
L180
L90
E3
N1
L90
F30
E4
N2
E2
F76
R90
W4
S3
R90
F2
L90
S3
R180
N5
E4
L90
W4
F1
N2
E1
F8
R90
F88
R180
F60
W2
R90
E3
N3
W5
F56
S1
E5
F5
L90
E4
N3
R90
E2
F34
W4
L90
F100
W4
L90
F40
L90
F51
E5
F52
N1
F45
W2
N2
F56
N2
W3
R180
F14
N3
L90
N2
F18
L180
E1
R90
N2
E3
L180
S5
F87
L90
F32
E1
F92
N1
W3
F89
E2
N1
R90
W1
F9
E1
F74
S1
L270
S1
F99
L90
W1
R90
F78
L90
W4
S2
R180
E3
S4
L90
F78
E5
L90
S5
W2
R90
N1
E5
F33
W1
R180
S1
W4
N1
F69
S5
R90
N5
F89
L90
W1
F91
L90
F19
E5
L90
F53
L90
S5
L90
S4
W1
S2
L180
F3
N5
N5
F78
E3
S1
L180
F79
L90
W4
R180
W3
N4
W5
F84
S4
L180
S1
S3
E2
S4
R90
N1
E5
S4
W4
R90
F44
R90
E5
S5
W1
N4
F37
N2
F41
R90
F58
L90
F5
R90
W4
L90
F45
N4
F48
S1
E2
S1
R90
F30
W2
L90
F53
L90
W5
R90
N2
E1
S3
F29
N5
L270
S2
F87
S4
F86
S4
R90
W5
F59
N2
F35
L90
W5
N3
E3
L90
S2
E2
N2
L90
R90
N5
L270
N5
R90
N4
E2
S3
W2
F55
E4
S1
L90
W3
S4
F95
W5
E2
R90
S3
F54
L90
N5
F69
R90
N1
W3
N4
F49
N4
E